# Acquisition Models for MR, PET and CT
This demonstration shows how to set-up and use SIRF/CIL acquisition models for different modalities. You should have tried the `introduction` notebook first. The current notebook briefly repeats some items without explanation.

This demo is a jupyter notebook, i.e. intended to be run step by step.
You could export it as a Python file and run it one go, but that might
make little sense as the figures are not labelled.


Authors: Christoph Kolbitsch, Edoardo Pasca, Kris Thielemans

First version: 23rd of April 2021  

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2019, 2021 University College London.   
Copyright 2021 Physikalisch-Technische Bundesanstalt.

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging
(http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

# Initial set-up

In [ ]:
# Make sure figures appears inline and animations works
%matplotlib widget
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('Introductory', 'acquisition_model_mr_pet_ct')

In [ ]:
# Initial imports etc
import numpy
import matplotlib.pyplot as plt

import os
import sys
import shutil
import brainweb
from tqdm.auto import tqdm

# Import MR, PET and CT functionality
import sirf.Gadgetron as mr
import sirf.STIR as pet
import cil.framework as ct

from sirf.Utilities import examples_data_path

Let's see if your installation comes with the ASTRA plugin. If not, we won't be able to illustrate CT forward projections via CIL unfortunately.

In [ ]:
try:
    from cil.plugins.astra.operators import ProjectionOperator as ap
    have_astra = True
except:
    have_astra = False
    print("CIL ASTRA plugin is not installed")

# Utilities

First define some handy function definitions to make subsequent code cleaner. You can ignore them when you first see this demo.
They have (minimal) documentation using Python docstrings such that you can do for instance `help(plot_2d_image)`

In [ ]:
def plot_2d_image(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot to plot 2D image"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar(shrink=.5)
    plt.title(title)
    plt.axis("off")

def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (numpy.array(vol.shape) - numpy.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    
    # Make a copy of the template to ensure we do not overwrite it
    templ_im_out = templ_im.copy()
    
    # Fill image content 
    templ_im_out.fill(numpy.reshape(vol, idim_orig))
    return(templ_im_out)

# Get brainweb data

We will download and use data from the brainweb. We will use a FDG image for PET and the PET uMAP for CT. MR usually provides qualitative images with an image contrast proportional to difference in T1, T2 or T2* depending on the sequence parameters. Nevertheless, we will make our life easy, by directly using the T1 map provided by the brainweb for MR.

In [ ]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

In [ ]:
for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(f, petNoise=1, t1Noise=0.75, t2Noise=0.75, petSigma=1, t1Sigma=1, t2Sigma=1)

In [ ]:
FDG_arr  = vol['PET']
T1_arr   = vol['T1']
uMap_arr = vol['uMap']

In [ ]:
# Display it
plt.figure();
slice_show = FDG_arr.shape[0]//2
plot_2d_image([1,3,1], FDG_arr[slice_show, 100:-100, 100:-100], 'FDG', cmap="hot")
plot_2d_image([1,3,2], T1_arr[slice_show, 100:-100, 100:-100], 'T1', cmap="Greys_r")
plot_2d_image([1,3,3], uMap_arr[slice_show, 100:-100, 100:-100], 'uMap', cmap="bone")

# Acquisition Models

In SIRF and CIL, an `AcquisitionModel` basically contains everything we need to know in order to describe what happens when we go from the imaged object to the acquired raw data (`AcquisitionData`) and then to the reconstructed image (`ImageData`). What we actually need to know depends strongly on the modality we are looking at. 

Here are some examples of modality specific information:  

  * __PET__: scanner geometry, detector efficiency, attenuation, randoms/scatter background...  
  * __CT__: scanner geometry  
  * __MR__: k-space sampling pattern, coil sensitivity information,...

and then there is information which is independent of the modality such as field-of-view or image discretisation (e.g. voxel sizes).

For __PET__ and __MR__ a lot of this information is already in the raw data. Because it would be quite a lot of work to enter all the necessary information by hand and then checking it is consistent, we create `AcquisitionModel` objects from `AcquisitionData` objects. The `AcquisitionData` only serves as a template and both its actual image and raw data content can be (and in this exercise will be) replaced. For __CT__ we will create an acquisition model from scratch, i.e. we will define the scanner geometry, image dimensions and image voxels sizes and so by hand.

So let's get started with __MR__ .

## MR

For MR we basically need the following:

  1. create an MR `AcquisitionData` object from a raw data file
  2. calculate the coil sensitivity maps (csm, for more information on that please see the notebook `MR/c_coil_combination.ipynb`). 
  3. then we will carry out a simple image reconstruction to get a `ImageData` object which we can use as a template for our `AcquisitionModel`
  4. then we will set up the MR `AcquisitionModel`

In [ ]:
# 1. create MR AcquisitionData
mr_acq = mr.AcquisitionData(os.path.join(examples_data_path('MR'),'grappa2_1rep.h5'))

In [ ]:
# 2. calculate CSM
preprocessed_data = mr.preprocess_acquisition_data(mr_acq)

csm = mr.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(preprocessed_data)

In [ ]:
# 3. calculate image template
recon = mr.FullySampledReconstructor()
recon.set_input(preprocessed_data)
recon.process()
im_mr = recon.get_output()

In [ ]:
# 4. create AcquisitionModel
acq_mod_mr = mr.AcquisitionModel(preprocessed_data, im_mr)

# Supply csm to the acquisition model 
acq_mod_mr.set_coil_sensitivity_maps(csm)

## PET

For PET we need to:

   1. create a PET `AcquisitionData` object from a raw data file
   2. create a PET `ImageData` object from the PET `AcquisitionData`
   3. then we will set up the PET `AcquisitionModel`

In [ ]:
# 1. create PET AcquisitionData
templ_sino = pet.AcquisitionData(os.path.join(examples_data_path('PET'),"thorax_single_slice","template_sinogram.hs"))

In [ ]:
# 2. create a template PET ImageData
im_pet = pet.ImageData(templ_sino)

In [ ]:
# 3. create AcquisitionModel

# create PET acquisition model
acq_mod_pet = pet.AcquisitionModelUsingRayTracingMatrix()
acq_mod_pet.set_up(templ_sino, im_pet)

## CT

For CT we need to:

   1. create a CT `AcquisitionGeometry` object
   2. obtain CT `ImageGeometry` from `AcquisitionGeometry`
   3. create a CT `ImageData` object
   4. then we will set up the CT `AcquisitionModel`

In [ ]:
# 1. define AcquisitionGeometry
angles = numpy.linspace(0, 360, 50, True, dtype=numpy.float32)
ag2d = ct.AcquisitionGeometry.create_Cone2D((0,-1000), (0, 500))\
          .set_panel(128,pixel_size=3.104)\
          .set_angles(angles)

In [ ]:
# 2. get ImageGeometry
ct_ig = ag2d.get_ImageGeometry()

In [ ]:
# 3. create ImageData
im_ct = ct_ig.allocate(None)

In [ ]:
# 4. create AcquisitionModel if we have ASTRA
if have_astra:
    acq_mod_ct = ap(ct_ig, ag2d, device='cpu')

# Apply acquisition models

## ImageData

In order to be able to apply our acquisition models in order to create raw data, we first need some image data. Because this image data has to fit to the `ImageData` and `AcquisitionData` objects of the different modalities, we will use them to create our image data. For more information on that please have a look at the notebook _introductory/introduction.ipynb_.

Let's create an `ImageData` object for each modality and display the slice in the centre of each data set:

In [ ]:
# MR
im_mr = crop_and_fill(im_mr, T1_arr)

# PET
im_pet = crop_and_fill(im_pet, FDG_arr)

# CT
im_ct = crop_and_fill(im_ct, uMap_arr)

plt.figure();
plot_2d_image([1,3,1], im_pet.as_array()[im_pet.dimensions()[0]//2, :, :], 'PET', cmap="hot")
plot_2d_image([1,3,2], numpy.abs(im_mr.as_array())[im_mr.dimensions()[0]//2, :, :], 'MR', cmap="Greys_r")
plot_2d_image([1,3,3], numpy.abs(im_ct.as_array()), 'CT', cmap="bone")

## Forward and Backward

### Fun fact
The methods `forward` and `backward` for __MR__ and __PET__ describe to forward acquisition model (i.e. going from the object to the raw data) and backward acquisition model (i.e. going from the raw data to the object). For the __CT__ acquisition model we utilise functionality from __CIL__ . Here, these two operations are defined by `direct` (corresponding to `forward`) and `adjoint` (corresponding to `backward`). In order to make sure that the __MR__ and __PET__ acquisition models are fully compatible with all the __CIL__ functionality, both acquisition models also have the methods `direct` and `adjoint` which are simple aliases of `forward` and `backward`. 

Now back to our three acquisition models and let's create some raw data

In [ ]:
# PET
raw_pet = acq_mod_pet.forward(im_pet)

# MR
raw_mr = acq_mod_mr.forward(im_mr)

# CT
if have_astra:
    raw_ct = acq_mod_ct.direct(im_ct)

and we can apply the backward/adjoint operation to do a simply image reconstruction.

In [ ]:
# PET
bwd_pet = acq_mod_pet.backward(raw_pet)

# MR
bwd_mr = acq_mod_mr.backward(raw_mr)

# CT
if have_astra:
    bwd_ct = acq_mod_ct.adjoint(raw_ct)

In [ ]:
plt.figure();
# Raw data
plot_2d_image([2,3,1], raw_pet.as_array()[0, raw_pet.dimensions()[1]//2, :, :], 'PET raw', cmap="viridis")
plot_2d_image([2,3,2], numpy.log(numpy.abs(raw_mr.as_array()[:, raw_mr.dimensions()[1]//2, :])), 'MR raw', cmap="viridis")
if have_astra:
        plot_2d_image([2,3,3], raw_ct.as_array(), 'CT raw', cmap="viridis")

# Rec data
plot_2d_image([2,3,4], bwd_pet.as_array()[bwd_pet.dimensions()[0]//2, :, :], 'PET', cmap="magma")
plot_2d_image([2,3,5], numpy.abs(bwd_mr.as_array()[bwd_mr.dimensions()[0]//2, :, :]), 'MR', cmap="Greys_r")
if have_astra:
    plot_2d_image([2,3,6], bwd_ct.as_array(), 'CT', cmap="bone")

These images don't look too great. This is due to many things, e.g. for MR the raw data is missing a lot of k-space information and hence we get undersampling artefacts.In general, the adjoint operation is not equal to the inverse operation. Therefore SIRF offers a range of more sophisticated image reconstruction techniques which strongly improve the image quality, but this is another notebook...